### Versão

In [150]:
versão = 1.0 # Disponíveis 1.0 e 2.0 (é só mudar o valor)

In [151]:
import sqlite3
import pandas as pd
import pyautogui
import time

### Métodos Utils

In [152]:
def V2_ExtrairPrimeiraPalavra(produtos):
    lista = []

    for _, row in produtos.iterrows():
        valor = row['Descrição'][0]
        valor = valor.split()
        valor = valor[0]
        lista.append(valor)

    lista_final = list(set(lista))
    
    return lista_final

def V2_AnalisarItens(produtos, data, banco):
    resultAnalise = []

    for index, row in produtos.iterrows():
        atual = produtos['Descrição'][index]
        atualPalavaras = atual.split()


        # VERIFICAÇÃO DE CNAE -------------------------------------------
        atualCNPJ = produtos['Cnpj'][index]
        query = f"SELECT * FROM CNPJcnae WHERE cnpj = {atualCNPJ}"
        datacnaes = pd.read_sql(query, banco)
        
        cnae = 'GERAL'
        if 'CONSTR' in datacnaes['atividade']:
            cnae = 'CONSTR'
        if ' AUTO' in datacnaes['atividade']:
            cnae = 'AUTO'

        dataFiltred = data.loc[data['hash'] == atualPalavaras[0]]
        dataFiltred = data.loc[data['segmentacao'] == cnae]

        # VERIFICAÇÃO DE INT --------------------------------------------


        
        grupoGT = 0

    return resultAnalise

### Configurações Versão 1.0

In [153]:
if versão == 1.0:
    excelBase = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSBePtdCMhpqTo83vz1h3xHFZTVuB16hnmXm-EP6y4nEix2bGDdODPkQGMYf3y7bhD64ZfY0B80g-OV/pub?output=xlsx'
    tempoEntreAcoes = 0.75

    préviaAnalise = True # False inicia a macro, True só exibe o número de grupos encontrados


### Configurações Versão 2.0

In [154]:
if versão == 2.0:
    xls = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRSkbeQ2LKKlmNmJfYp_ijFT8vRM7Nd2_WqMGoYsmylGnnZWVqhEIB7FA2J5gmxBaEkjvd0Qflc0_sr/pub?output=xlsx'
    banco = sqlite3.connect('fiote.db')
    cursor = banco.cursor()

### Código Principal Versão 1.0

In [182]:
if versão == 1.0:

    inicio_time = time.time()

    # Leitura Excel
    data = pd.read_excel(excelBase,  sheet_name="Cerebelo")
    produtos = pd.read_excel(excelBase, sheet_name="Input")
    data.drop_duplicates()

    fim_leituraExcel = time.time()
    tempoLeitura = fim_leituraExcel - inicio_time
    
    #Lista para gerar Excel
    excelProdutosAnalisado = []
    excelHashValue = []
    excelNC = []

    resultado_GP = []

    for index, row in produtos.iterrows():
        produtoAtual = produtos["Descrição"][index].upper()
        produtoPalavras = produtoAtual.split(' ')
        
        excelProdutosAnalisado.append(produtoAtual)

        # Inicializa GP e flags
        GP = 0
        encontrou_correspondencia = False

        # Filtrar os dados apenas pelo Hash
        data_filtred = data.loc[data['Hash'] == produtoPalavras[0]]

        for index_, row in data_filtred.iterrows():
            value = str(row['Value'])

            # Se o valor contiver ";", divida-o em palavras e verifique se todas estão presentes em produtoAtual
            if ';' in value:
                palavras_value = value.split(';')
                # Verifica se todas as palavras estão presentes em produtoAtual, incluindo os espaços exatos
                if all(palavra in produtoAtual for palavra in palavras_value):
                    GP = row['GP']
                    break
            else:
                # Se não houver ";", verifica se o valor completo está presente em produtoAtual
                if value in produtoAtual:
                    GP = row['GP']
                    break


        if GP != 0:
            value_nc = str(row['NC'])
            if ';' in value_nc:
                palavras_value_nc = value_nc.split(';')
                # Verifica palavra exata, incluindo espaços
                if any(palavra in produtoAtual for palavra in palavras_value_nc):
                    GP = 0
                    palavra_bloqueio = next(
                        palavra for palavra in palavras_value_nc if palavra in produtoAtual
                    )
                    excelHashValue.append(f"{value}")
                    excelNC.append(palavra_bloqueio)
                    encontrou_correspondencia = True
            else:
                # Busca por value_nc exato, incluindo espaços
                if value_nc in produtoAtual:
                    GP = 0
                    palavra_bloqueio = value_nc
                    excelHashValue.append(f"{value}")
                    excelNC.append(palavra_bloqueio)
                    encontrou_correspondencia = True


        if not encontrou_correspondencia:
            excelHashValue.append(f"{value if GP != 0 else '-'}")
            excelNC.append("-")
        
        resultado_GP.append(int(GP))

    fim_processamento = time.time()
    tempoprocessamento = fim_processamento - fim_leituraExcel

    df_PlanilhaExcel = pd.DataFrame({
        'Grupo': resultado_GP,
        'Produto': excelProdutosAnalisado,
        'Hash e Value': excelHashValue,
        'NC': excelNC
    })

    df_PlanilhaExcel.to_excel('ResultadoAnalise.xlsx', index=False)

    fim_gerarPlanilha = time.time()
    tempoGerarPlanilha = fim_gerarPlanilha - fim_processamento

    tempoExecuçãoTotal = fim_gerarPlanilha - inicio_time

    if préviaAnalise:
        print(f"------------------------------ TEMPO DE EXECUCAO ------------------------------")
        print()
        print(f"CARREGAR BASE DE DADOS: {tempoLeitura:.2f}s")
        print(f"PROCESSAR PRODUTOS: {tempoprocessamento:.2f}s")
        print(f"GERAR PLANILHA: {tempoGerarPlanilha:.2f}s")
        print()
        print("--------------------------------------------------------------------------------")
        print(f"------------------------------------------------------------------ TOTAL: {tempoExecuçãoTotal:.2f}")
        print("--------------------------------------------------------------------------------")
        print()
        print()

        print(f"------------------------------ ANALISE DETALHADA ------------------------------")
        print()
        print(f"PRODUTOS NAO VINCULADOS: {len([val for val in resultado_GP if val == 0])}i")
        print(f"PRODUTOS VINCULADOS: {len([val for val in resultado_GP if val != 0])}i")
        print(f"GRUPOS DIFFS: {len(set([val for val in resultado_GP if val != 0]))}g")
        print()
        print("--------------------------------------------------------------------------------")
        print(f"------------------------------------------------------------------ TOTAL:  {len(resultado_GP)}")
        print("--------------------------------------------------------------------------------")
        print()
        print()
    else:

        pyautogui.PAUSE = tempoEntreAcoes                                     
        pyautogui.hotkey('alt', 'tab')

        linha = 0
        indiceFinal = 0
        PressionarBotaoIndiceVezes = False
        BuscarIndiceFinal = False

        # Novo loop baseado em resultado_GP
        for indice, gp_value in enumerate(resultado_GP):
            indiceatual = indice

            if indiceFinal == indice:
                BuscarIndiceFinal = True

            if gp_value == 0:
                
                while BuscarIndiceFinal:
                    valor = resultado_GP[indice]

                    if valor != 0:
                        indiceFinal = indice
                        BuscarIndiceFinal = False
                        PressionarBotaoIndiceVezes = True
                        break

                    indice += 1

            else:
                
                pyautogui.press('f12')
                pyautogui.write(str(valor))
                pyautogui.press('enter')
                pyautogui.press('down', presses=10)
                pyautogui.press('enter')
                time.sleep(1.4)
                pyautogui.press('right')
                pyautogui.press('enter')
                pyautogui.click(957,627)
                pyautogui.click(642,241)
                pyautogui.press('down', presses= linha + 1)
                time.sleep(1.2)

            if PressionarBotaoIndiceVezes:
                pyautogui.press('Down', presses=indiceFinal - indiceatual)
                time.sleep(1.2)
                PressionarBotaoIndiceVezes = False

            linha += 1

------------------------------ TEMPO DE EXECUCAO ------------------------------

CARREGAR BASE DE DADOS: 16.76s
PROCESSAR PRODUTOS: 1.31s
GERAR PLANILHA: 0.20s

--------------------------------------------------------------------------------
------------------------------------------------------------------ TOTAL: 18.27
--------------------------------------------------------------------------------


------------------------------ ANALISE DETALHADA ------------------------------

PRODUTOS NAO VINCULADOS: 1545i
PRODUTOS VINCULADOS: 149i
GRUPOS DIFFS: 3g

--------------------------------------------------------------------------------
------------------------------------------------------------------ TOTAL:  1694
--------------------------------------------------------------------------------




### Código Principal Versão 2.0 (em desenvolvimento)

In [156]:
if versão == 2.0:
    
    if True == False: #Mude o 2º 'False' para 'True' afim de fazer o backupLines (v2.0: do excel para o banco)
        dataBackup = pd.read_excel(xls, sheet_name='BackupLines')

        for index, row in dataBackup.iterrows():
            listColunas = list(dataBackup.columns)
            strColunas = ', '.join(listColunas)
            valores = list(row.values)

            parametros = ', '.join(['?'] * len(listColunas))

            query = f"INSERT INTO VPcerebelo ({strColunas}) VALUES ({parametros})"

            cursor.execute(query, valores)
    banco.commit()

    produtos = pd.read_excel(xls, sheet_name='Input')
    consultaSQL_palavras = V2_ExtrairPrimeiraPalavra(produtos)

    parametros = ', '.join(['?'] * len(consultaSQL_palavras))
    query = f"SELECT id, gp, hash, value, excecao, segmentacao, type FROM VPcerebelo WHERE hash IN ({parametros})"
    data = pd.read_sql(query, banco, params=consultaSQL_palavras)

    resultAnalise = V2_AnalisarItens(produtos, data, banco)

    banco.close


